# Piru Music Composer - Composer only

This notebook can be run in parallel of a training to check the output produced by an intermediate epoch

In [119]:
import guitarpro
import pandas as pd
import numpy as np
import math
import os
from music21 import *

In [120]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint

model_path = 'output/weights-improvement-08-0.5145-bigger.hdf5'
print(model_path)

model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_features))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

model.load_weights(model_path)

output/weights-improvement-08-0.5145-bigger.hdf5
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 100, 256)          537600    
_________________________________________________________________
dropout_28 (Dropout)         (None, 100, 256)          0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 100, 512)          1574912   
_________________________________________________________________
dropout_29 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
lstm_30 (LSTM)               (None, 256)               787456    
_________________________________________________________________
dense_19 (Dense)             (None, 256)               65792     
_________________________________________________________________
drop

In [121]:
variety = 0
# We impose minimum 6 different notes or chords
while variety < 6:
    start = np.random.randint(0, len(network_input)-1)
    mixup = 10
    pattern = network_input[start,:mixup]
    for i in range(1,10):
        pattern = np.append(pattern,network_input[start*(i+1)%len(network_input),:mixup],axis=0)
        
    prediction_output = []
    
    # generate 100 notes or chords
    for note_index in range(100):
        prediction_input = np.reshape(pattern, (1, pattern.shape[0], pattern.shape[1]))
        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        prediction_output+=[onehotenc_notes.columns[index]]
        prediction[:,index]=1
        prediction[prediction<1]=0
        pattern = np.append(pattern,prediction.astype('uint8'), axis=0)
        pattern = pattern[1:]

    variety = len(set(prediction_output))
    print('variety',variety,set(prediction_output))
print(prediction_output)

offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.ElectricGuitar()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    elif pattern == 'x':
        new_note = note.Note(999)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Woodblock()
        output_notes.append(new_note)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.ElectricGuitar()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='output/test_output.mid')


variety 5 {'61.56.49', '60.55.48', '64.59.52', 'x', '59.54.47'}
variety 2 {'64.59.52', '58.53.46'}
variety 5 {'45', '55.50.45', '62.57.50', '48', '47'}
variety 7 {'57.52.45', '63.58.51', '55.50.43', '60.55.48', '56.51.44', '45.40', 'x'}
['63.58.51', '63.58.51', '63.58.51', '57.52.45', '57.52.45', 'x', '57.52.45', '56.51.44', '55.50.43', '45.40', '57.52.45', '57.52.45', 'x', '57.52.45', '56.51.44', '55.50.43', '45.40', '60.55.48', '60.55.48', '60.55.48', '60.55.48', '60.55.48', '60.55.48', '63.58.51', '63.58.51', '63.58.51', '63.58.51', '63.58.51', '63.58.51', '57.52.45', '57.52.45', 'x', '57.52.45', '56.51.44', '55.50.43', '45.40', '57.52.45', '57.52.45', 'x', '57.52.45', '56.51.44', '55.50.43', '45.40', '60.55.48', '60.55.48', '60.55.48', '60.55.48', '60.55.48', '60.55.48', '63.58.51', '63.58.51', '63.58.51', '63.58.51', '63.58.51', '63.58.51', '57.52.45', '57.52.45', 'x', '57.52.45', '56.51.44', '55.50.43', '45.40', '57.52.45', '57.52.45', 'x', '57.52.45', '56.51.44', '55.50.43', '45

'output/test_output.mid'